# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
import pandas as pd
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
#sales = graphlab.SFrame('kc_house_data.gl/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [3]:
train_data = pd.read_csv('kc_house_train_data.csv')
test_data = pd.read_csv('kc_house_test_data.csv')

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
from sklearn.linear_model import LinearRegression

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = LinearRegression()
example_model.fit(train_data[example_features],train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [6]:
intercept = example_model.intercept_
coef = example_model.coef_

In [7]:
pd.DataFrame({'name':['intercept']+example_features,'value':[intercept]+list(coef)})

,name,value
0,intercept,87912.865815
1,sqft_living,315.406691
2,bedrooms,-65081.887116
3,bathrooms,6942.165986


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [8]:
example_predictions = example_model.predict(train_data[example_features])
print example_predictions[0] # should be 271789.505878

271789.26538


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [9]:
def get_residual_sum_of_squares(model, data, outcome,features=['sqft_living', 'bedrooms', 'bathrooms']):
    # First get the predictions
    predictions = model.predict(data[features])
    # Then compute the residuals/errors
    errors = predictions - outcome
    # Then square and add them up
    RSS = np.square(errors).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [10]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761940583e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [11]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [12]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [13]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x:log(x))
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x:log(x))
test_data['lat_plus_long'] = test_data['lat'] + train_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [14]:
test_data[['bedrooms_squared','bed_bath_rooms','log_sqft_living','lat_plus_long']].mean()

bedrooms_squared    12.446678
bed_bath_rooms       7.503902
log_sqft_living      7.550275
lat_plus_long      -74.653175
dtype: float64

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [15]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [16]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = LinearRegression()
model_1.fit(train_data[model_1_features],train_data['price'])
model_2 = LinearRegression()
model_2.fit(train_data[model_2_features],train_data['price'])
model_3 = LinearRegression()
model_3.fit(train_data[model_3_features],train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
# Examine/extract each model's coefficients:
for model,features in zip([model_1,model_2,model_3],[model_1_features,model_2_features,model_3_features]):
    intercept = model.intercept_
    coef = model.coef_
    print pd.DataFrame({'name':['intercepte']+features,'value':[intercept]+list(coef)})

          name         value
0   intercepte -6.907573e+07
1  sqft_living  3.122586e+02
2     bedrooms -5.958653e+04
3    bathrooms  1.570674e+04
4          lat  6.586193e+05
5         long -3.093744e+05
             name         value
0      intercepte -6.686797e+07
1     sqft_living  3.066101e+02
2        bedrooms -1.134464e+05
3       bathrooms -7.146131e+04
4             lat  6.548446e+05
5            long -2.942990e+05
6  bed_bath_rooms  2.557965e+04
               name         value
0        intercepte -6.203608e+07
1       sqft_living  5.294228e+02
2          bedrooms  3.451423e+04
3         bathrooms  6.706078e+04
4               lat  5.340856e+05
5              long -4.067507e+05
6    bed_bath_rooms -8.570504e+03
7  bedrooms_squared -6.788587e+03
8   log_sqft_living -5.618315e+05
9     lat_plus_long  1.273349e+05


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [18]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
for model,features in zip([model_1,model_2,model_3],[model_1_features,model_2_features,model_3_features]):
    print get_residual_sum_of_squares(model,train_data,train_data['price'],features)

9.6787996305e+14
9.58419635074e+14
9.0343645505e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [19]:
# Compute the RSS on TESTING data for each of the three models and record the values:
for model,features in zip([model_1,model_2,model_3],[model_1_features,model_2_features,model_3_features]):
    print get_residual_sum_of_squares(model,test_data,test_data['price'],features)

2.25500469795e+14
2.23377462976e+14
2.61942651061e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.